# MRP

## Auftragsabrufe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

orders = pd.DataFrame([
    ["A1", pd.to_datetime("14/11/2025", dayfirst=True), "P1", 5],
    ["A2", pd.to_datetime("13/11/2025", dayfirst=True), "P3", 3],
    ["A3", pd.to_datetime("19/11/2025", dayfirst=True), "P2", 4],
    ["A4", pd.to_datetime("25/11/2025", dayfirst=True), "P4", 2],
    ["A5", pd.to_datetime("04/12/2025", dayfirst=True), "P5", 3],
    ["A6", pd.to_datetime("03/12/2025", dayfirst=True), "P1", 4],
], columns=["AuftragId", "Liefertermin", "ProduktId", "Menge"])
orders['Frist'] = (orders['Liefertermin'] - pd.to_datetime("09/11/2025", dayfirst=True)) / np.timedelta64(1, 'h')
print(orders.dtypes)
orders

AuftragId               object
Liefertermin    datetime64[ns]
ProduktId               object
Menge                    int64
Frist                  float64
dtype: object


,AuftragId,Liefertermin,ProduktId,Menge,Frist
0,A1,2025-11-14,P1,5,120.0
1,A2,2025-11-13,P3,3,96.0
2,A3,2025-11-19,P2,4,240.0
3,A4,2025-11-25,P4,2,384.0
4,A5,2025-12-04,P5,3,600.0
5,A6,2025-12-03,P1,4,576.0


## Aufträge

In [2]:
orders = [
    ("A1", "P1", 5, 40),
    ("A2", "P3", 3, 32),
    ("A3", "P2", 4, 56),
    ("A4", "P4", 2, 72),
    ("A5", "P5", 3, 96),
    ("A6", "P1", 4, 88),
]

## BOM

In [3]:
bom = [
    # Parent, Component, Menge pro Parent
    ("P1", "RohrA", 2),
    ("P1", "RohrB", 1),
    ("P1", "PlatteC", 1),
    ("P2", "BlechD", 1),
    ("P2", "LascheE", 2),
    ("P3", "PlatteF", 2),
    ("P3", "BolzenG", 2),
    ("P4", "RohrH", 1),
    ("P4", "FlanschI", 2),
    ("P4", "PlatteJ", 1),
    ("P5", "WandK", 2),
    ("P5", "DeckelL", 1),
    ("P5", "VerstärkungM", 2),
]


## Arbeitspläne

In [4]:
routings = {
    "P1": [("M9_Schweißen", "MAG-Schweißen", 2.5, 0.75)],
    "P2": [("M9_Schweißen", "WIG-Schweißen", 2.0, 0.75)],
    "P3": [("M9_Schweißen", "MAG-Schweißen", 1.8, 0.75)],
    "P4": [("M9_Schweißen", "Lichtbogen", 2.2, 0.75)],
    "P5": [("M9_Schweißen", "WIG-Schweißen", 2.8, 0.75)],

    "RohrA": [("M1_Säge", "Zuschnitt", 0.3, 0.25), ("M2_Schleifer", "Entgraten", 0.2, 0.1)],
    "RohrB": [("M1_Säge", "Zuschnitt", 0.4, 0.25), ("M2_Schleifer", "Entgraten", 0.25, 0.1)],
    "PlatteC": [("M3_Brenner", "Brennschneiden", 0.5, 0.3), ("M4_Bohrmaschine", "Bohren", 0.4, 0.2)],
    "BlechD": [("M5_Laser", "Laserschneiden", 0.6, 0.4), ("M6_Abkantpresse", "Kanten", 0.5, 0.3)],
    "LascheE": [("M5_Laser", "Laserschneiden", 0.4, 0.4), ("M4_Bohrmaschine", "Bohren", 0.3, 0.2)],
    "PlatteF": [("M1_Säge", "Zuschnitt", 0.4, 0.25), ("M4_Bohrmaschine", "Bohren", 0.3, 0.2)],
    "BolzenG": [("M7_Drehmaschine", "Drehen", 0.6, 0.3), ("M8_Fräse", "Fräsen", 0.4, 0.25)],
    "RohrH": [("M1_Säge", "Sägen", 0.5, 0.25), ("M4_Bohrmaschine", "Bohren", 0.4, 0.2)],
    "FlanschI": [("M3_Brenner", "Brennschneiden", 0.5, 0.3), ("M4_Bohrmaschine", "Bohren", 0.3, 0.2)],
    "PlatteJ": [("M5_Laser", "Laserschneiden", 0.7, 0.4), ("M2_Schleifer", "Entgraten", 0.2, 0.1)],
    "WandK": [("M5_Laser", "Blechzuschnitt", 0.8, 0.4), ("M6_Abkantpresse", "Kanten", 0.6, 0.3)],
    "DeckelL": [("M1_Säge", "Zuschnitt", 0.4, 0.25), ("M4_Bohrmaschine", "Bohren", 0.3, 0.2)],
    "VerstärkungM": [("M7_Drehmaschine", "Drehen", 0.7, 0.3), ("M8_Fräse", "Fräsen", 0.5, 0.25)],
}


## Maschinenkapazität

In [5]:
machine_capacity = {
    "M1_Säge": 40,   # Stunden pro Woche
    "M2_Schleifer": 40,
    "M3_Brenner": 40,
    "M4_Bohrmaschine": 40,
    "M5_Laser": 40,
    "M6_Abkantpresse": 40,
    "M7_Drehmaschine": 40,
    "M8_Fräse": 40,
    "M9_Schweißen": 40,
}


## Berechnung

### Hilfsfunktionen

In [6]:
def total_lead_time(part, quantity, routings):
    """Summe aus Bearbeitungszeit + Rüstzeit je Vorgang"""
    if part not in routings:
        return 0.0
    total = 0
    for machine, op, dur, setup in routings[part]:
        total += setup + dur * quantity
    return total

def explode_bom(product, quantity, bom):
    """Gibt Unterteile und Mengen für ein Produkt zurück"""
    return [(comp, quantity * qty) for parent, comp, qty in bom if parent == product]

### MRP

In [7]:
from collections import deque

def explode_bom(product, quantity, bom, verbose=False):
    """
    Gibt alle Unterteile (Childs) für ein bestimmtes Produkt zurück.
    Multipliziert die Mengen entsprechend der benötigten Produktmenge.
    
    Args:
        product (str): Produktname oder ID (Parent in der BOM)
        quantity (float): Menge des Produkts, für die Unterteile berechnet werden sollen
        bom (list of tuples): [(Parent, Component, Menge_pro_Parent), ...]
        verbose (bool): Wenn True, werden Debug-Informationen ausgegeben

    Returns:
        list of tuples: [(Component, benötigte_Menge), ...]
    """
    children = []

    if verbose:
        print(f"\n🔍 Explodiere Stückliste für Produkt: {product}, Menge: {quantity}")

    for parent, comp, qty in bom:
        if parent == product:
            child_qty = quantity * qty
            children.append((comp, child_qty))
            if verbose:
                print(f"  ➜ {product} benötigt {qty}× {comp} → {child_qty} Einheiten")

    if not children and verbose:
        print(f"  ⚠️ Keine Unterteile für {product} gefunden (evtl. Endteil).")

    return children

from collections import deque

def run_mrp(orders, bom, routings):
    mrp = []  # Liste aller Bedarfe mit Terminen
    
    queue = deque()
    for order_id, product, qty, due in orders:
        queue.append((order_id, product, qty, due, "Endprodukt"))
    
    while queue:
        order_id, part, qty, need_date, parent = queue.popleft()
        print(f"\n📦 Verarbeite Teil: {part}, Menge: {qty}, Fällig: {need_date}, Parent: {parent}")
        
        # Durchlaufzeit berechnen und Starttermin rückrechnen
        lt = total_lead_time(part, qty, routings)
        start_date = need_date - lt
        
        mrp.append({
            "Order": order_id,
            "Parent": parent,
            "Part": part,
            "Qty": qty,
            "Need_Date": need_date,
            "Start_Date": round(start_date, 2),
            "Lead_Time": round(lt, 2)
        })
        
        children = explode_bom(part, qty, bom, verbose=True)

        # Unterteile auflösen
        for (child, child_qty) in children:
            print(f"   ➕ Füge {child} ({child_qty} Stk) zur Warteschlange hinzu (Parent: {part})")
            queue.append((order_id, child, child_qty, start_date, part))
    
    return mrp

run_mrp(orders, bom, routings)


📦 Verarbeite Teil: P1, Menge: 5, Fällig: 40, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P1, Menge: 5
  ➜ P1 benötigt 2× RohrA → 10 Einheiten
  ➜ P1 benötigt 1× RohrB → 5 Einheiten
  ➜ P1 benötigt 1× PlatteC → 5 Einheiten
   ➕ Füge RohrA (10 Stk) zur Warteschlange hinzu (Parent: P1)
   ➕ Füge RohrB (5 Stk) zur Warteschlange hinzu (Parent: P1)
   ➕ Füge PlatteC (5 Stk) zur Warteschlange hinzu (Parent: P1)

📦 Verarbeite Teil: P3, Menge: 3, Fällig: 32, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P3, Menge: 3
  ➜ P3 benötigt 2× PlatteF → 6 Einheiten
  ➜ P3 benötigt 2× BolzenG → 6 Einheiten
   ➕ Füge PlatteF (6 Stk) zur Warteschlange hinzu (Parent: P3)
   ➕ Füge BolzenG (6 Stk) zur Warteschlange hinzu (Parent: P3)

📦 Verarbeite Teil: P2, Menge: 4, Fällig: 56, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P2, Menge: 4
  ➜ P2 benötigt 1× BlechD → 4 Einheiten
  ➜ P2 benötigt 2× LascheE → 8 Einheiten
   ➕ Füge BlechD (4 Stk) zur Warteschlange hinzu (Parent:

[{'Order': 'A1',
  'Parent': 'Endprodukt',
  'Part': 'P1',
  'Qty': 5,
  'Need_Date': 40,
  'Start_Date': 26.75,
  'Lead_Time': 13.25},
 {'Order': 'A2',
  'Parent': 'Endprodukt',
  'Part': 'P3',
  'Qty': 3,
  'Need_Date': 32,
  'Start_Date': 25.85,
  'Lead_Time': 6.15},
 {'Order': 'A3',
  'Parent': 'Endprodukt',
  'Part': 'P2',
  'Qty': 4,
  'Need_Date': 56,
  'Start_Date': 47.25,
  'Lead_Time': 8.75},
 {'Order': 'A4',
  'Parent': 'Endprodukt',
  'Part': 'P4',
  'Qty': 2,
  'Need_Date': 72,
  'Start_Date': 66.85,
  'Lead_Time': 5.15},
 {'Order': 'A5',
  'Parent': 'Endprodukt',
  'Part': 'P5',
  'Qty': 3,
  'Need_Date': 96,
  'Start_Date': 86.85,
  'Lead_Time': 9.15},
 {'Order': 'A6',
  'Parent': 'Endprodukt',
  'Part': 'P1',
  'Qty': 4,
  'Need_Date': 88,
  'Start_Date': 77.25,
  'Lead_Time': 10.75},
 {'Order': 'A1',
  'Parent': 'P1',
  'Part': 'RohrA',
  'Qty': 10,
  'Need_Date': 26.75,
  'Start_Date': 21.4,
  'Lead_Time': 5.35},
 {'Order': 'A1',
  'Parent': 'P1',
  'Part': 'RohrB',
 

In [8]:
mrp_result = run_mrp(orders, bom, routings)

import pandas as pd
df_mrp = pd.DataFrame(mrp_result)
df_mrp.sort_values(["Order", "Start_Date"], inplace=True)
print(df_mrp.head(20))
df_mrp


📦 Verarbeite Teil: P1, Menge: 5, Fällig: 40, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P1, Menge: 5
  ➜ P1 benötigt 2× RohrA → 10 Einheiten
  ➜ P1 benötigt 1× RohrB → 5 Einheiten
  ➜ P1 benötigt 1× PlatteC → 5 Einheiten
   ➕ Füge RohrA (10 Stk) zur Warteschlange hinzu (Parent: P1)
   ➕ Füge RohrB (5 Stk) zur Warteschlange hinzu (Parent: P1)
   ➕ Füge PlatteC (5 Stk) zur Warteschlange hinzu (Parent: P1)

📦 Verarbeite Teil: P3, Menge: 3, Fällig: 32, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P3, Menge: 3
  ➜ P3 benötigt 2× PlatteF → 6 Einheiten
  ➜ P3 benötigt 2× BolzenG → 6 Einheiten
   ➕ Füge PlatteF (6 Stk) zur Warteschlange hinzu (Parent: P3)
   ➕ Füge BolzenG (6 Stk) zur Warteschlange hinzu (Parent: P3)

📦 Verarbeite Teil: P2, Menge: 4, Fällig: 56, Parent: Endprodukt

🔍 Explodiere Stückliste für Produkt: P2, Menge: 4
  ➜ P2 benötigt 1× BlechD → 4 Einheiten
  ➜ P2 benötigt 2× LascheE → 8 Einheiten
   ➕ Füge BlechD (4 Stk) zur Warteschlange hinzu (Parent:

,Order,Parent,Part,Qty,Need_Date,Start_Date,Lead_Time
6,A1,P1,RohrA,10,26.75,21.40,5.35
8,A1,P1,PlatteC,5,26.75,21.75,5.00
7,A1,P1,RohrB,5,26.75,23.15,3.60
0,A1,Endprodukt,P1,5,40.00,26.75,13.25
10,A2,P3,BolzenG,6,25.85,19.30,6.55
9,A2,P3,PlatteF,6,25.85,21.20,4.65
1,A2,Endprodukt,P3,3,32.00,25.85,6.15
12,A3,P2,LascheE,8,47.25,41.05,6.20
11,A3,P2,BlechD,4,47.25,42.15,5.10
2,A3,Endprodukt,P2,4,56.00,47.25,8.75


## Visualisierung

In [13]:
# Ablaufplanung
schedule = []

for _, order in df_mrp.iterrows():
    parent = order["Parent"]
    part = order["Part"]
    current_start = order["Start_Date"]

    # 1️⃣ zuerst: Routing des Parts (z. B. RohrA)
    for machine, operation, duration, setup in routings.get(part, []):
        start = current_start
        end = start + setup + duration
        schedule.append({
            "Order": order["Order"],
            "Item": part,
            "Machine": machine,
            "Operation": operation,
            "Start": start,
            "End": end
        })
        current_start = end

    # 2️⃣ danach: Routing des Parents (z. B. P1)
    for machine, operation, duration, setup in routings.get(parent, []):
        start = current_start
        end = start + setup + duration
        schedule.append({
            "Order": order["Order"],
            "Item": parent,
            "Machine": machine,
            "Operation": operation,
            "Start": start,
            "End": end
        })
        current_start = end

schedule_df = pd.DataFrame(schedule)
print(schedule_df)


   Order          Item          Machine       Operation  Start    End
0     A1         RohrA          M1_Säge       Zuschnitt  21.40  21.95
1     A1         RohrA     M2_Schleifer       Entgraten  21.95  22.25
2     A1            P1     M9_Schweißen   MAG-Schweißen  22.25  25.50
3     A1       PlatteC       M3_Brenner  Brennschneiden  21.75  22.55
4     A1       PlatteC  M4_Bohrmaschine          Bohren  22.55  23.15
5     A1            P1     M9_Schweißen   MAG-Schweißen  23.15  26.40
6     A1         RohrB          M1_Säge       Zuschnitt  23.15  23.80
7     A1         RohrB     M2_Schleifer       Entgraten  23.80  24.15
8     A1            P1     M9_Schweißen   MAG-Schweißen  24.15  27.40
9     A1            P1     M9_Schweißen   MAG-Schweißen  26.75  30.00
10    A2       BolzenG  M7_Drehmaschine          Drehen  19.30  20.20
11    A2       BolzenG         M8_Fräse          Fräsen  20.20  20.85
12    A2            P3     M9_Schweißen   MAG-Schweißen  20.85  23.40
13    A2       Platt

### Gantt

In [16]:
import plotly.express as px
import datetime as dt

base_date = dt.date(2025, 11, 1)
schedule_df["Start_dt"] = schedule_df["Start"].apply(lambda x: base_date + dt.timedelta(days=x))
schedule_df["End_dt"] = schedule_df["End"].apply(lambda x: base_date + dt.timedelta(days=x))


fig = px.timeline(
    schedule_df,
    x_start="Start_dt",
    x_end="End_dt",
    y="Machine",
    color="Item",
    text="Operation",
)

fig.update_yaxes(autorange="reversed")
fig.show()


## Kapazitätsbelastung und Angebot

In [17]:
import pandas as pd
import numpy as np
import datetime as dt


# 1️⃣ Umrechnung in "echte" Datumswerte
#base_date = dt.date(2025, 11, 1)
#schedule_df["Start_dt"] = schedule_df["Start"].apply(lambda x: base_date + dt.timedelta(days=x))
#schedule_df["End_dt"] = schedule_df["End"].apply(lambda x: base_date + dt.timedelta(days=x))
schedule_df["Duration_days"] = schedule_df["End"] - schedule_df["Start"]
schedule_df["Duration_hours"] = schedule_df["Duration_days"] * 24  # falls 1 Tag = 24h

# 2️⃣ Aufteilen in Tagesbelastung (z. B. 8h pro Arbeitstag)
records = []
for _, row in schedule_df.iterrows():
    current_day = row["Start_dt"]
    end_day = row["End_dt"]
    remaining_hours = row["Duration_hours"]

    while current_day < end_day:
        next_day = current_day + dt.timedelta(days=1)
        work_hours_today = min(8, remaining_hours)  # hier: max 8h/Tag angenommen
        records.append({
            "Machine": row["Machine"],
            "Date": current_day,
            "Load_h": work_hours_today
        })
        remaining_hours -= work_hours_today
        current_day = next_day

load_df = pd.DataFrame(records)

# 3️⃣ Gruppieren nach Maschine & Tag
machine_load = (
    load_df.groupby(["Machine", "Date"], as_index=False)["Load_h"].sum()
)

print(machine_load)


            Machine        Date  Load_h
0           M1_Säge  2026-01-04     8.0
1           M1_Säge  2026-01-12     8.0
2      M2_Schleifer  2025-11-22     7.2
3      M2_Schleifer  2025-11-24     8.0
4      M2_Schleifer  2026-01-14     8.0
5        M3_Brenner  2025-11-22     8.0
6   M4_Bohrmaschine  2025-11-22     8.0
7   M4_Bohrmaschine  2025-11-23     8.0
8   M4_Bohrmaschine  2025-12-12     8.0
9   M4_Bohrmaschine  2026-01-03     8.0
10  M4_Bohrmaschine  2026-01-13     8.0
11  M4_Bohrmaschine  2026-01-24     8.0
12         M5_Laser  2025-12-13     8.0
13         M5_Laser  2026-01-04     8.0
14         M5_Laser  2026-01-17     8.0
15  M6_Abkantpresse  2026-01-18     8.0
16  M7_Drehmaschine  2025-11-20     8.0
17  M7_Drehmaschine  2026-01-19     8.0
18     M9_Schweißen  2025-11-21     8.0
19     M9_Schweißen  2025-11-22     8.0
20     M9_Schweißen  2025-11-23    24.0
21     M9_Schweißen  2025-11-24    24.0
22     M9_Schweißen  2025-11-25    24.0
23     M9_Schweißen  2025-11-26    24.0


In [18]:
import plotly.express as px

fig = px.bar(
    machine_load,
    x="Date",
    y="Load_h",
    color="Machine",
    barmode="group",
    title="Maschinenbelastung pro Tag (h)"
)

# Optionale Kapazitätslinie (8h pro Tag oder 40h pro Woche)
fig.add_hline(y=8, line_dash="dot", annotation_text="Tageskapazität (8h)", annotation_position="top right")

fig.show()
